# Logical shadow tomography: $[[5, 1, 3]]$ code example

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

from context import *
from convert import *

In [ ]:
plt.rcParams.update({"font.family": "serif", "font.size": 16})

### Helper functions

In [ ]:
from typing import List, Union


@njit
def pauli_operation(gs_stb, ps_stb, gs_obs, r):
    """Apply a Pauli gate(gs_obs) to stabilizer state.

    Parameters:
    gs_stb: int (2*N, 2*N) - Pauli strings in original stabilizer tableau.
    ps_stb: int (N) - phase indicators of (de)stabilizers.
    gs_obs: int (L, 2*N) - strings of Pauli gates to apply from 0 to L-1.
    r: int - log2 rank of density matrix (num of standby stablizers).

    Returns:
    gs_stb: int (2*N, 2*N) - Pauli strings in updated stabilizer tableau.
    ps_stb: int (N) - phase indicators of (de)stabilizers.
    r: int - updated log2 rank of density matrix.
    """
    (L, Ng) = gs_obs.shape
    N = Ng // 2
    assert 0 <= r <= N
    for k in range(L):  # for each observable gs_obs[k]
        for j in range(2 * N):
            if utils.acq(
                gs_stb[j], gs_obs[k]
            ):  # find gs_stb[j] anticommute with gs_obs[k]
                if (
                    r <= j < N
                ):  # gs_stb[j] anti-commute with gs_obs[k] gate, flip the sign
                    ps_stb[j] = (ps_stb[j] + 2) % 4

    return gs_stb, ps_stb, r


def stoc_depolarize_map(stab_state, pvalues: Union[float, List[float]]):
    if not isinstance(pvalues, list):
        pvalues = [pvalues]
    if len(pvalues) == 1:
        px = py = pz = pvalues[0] / 3
    elif len(pvalues) == 3:
        px, py, pz = pvalues
    else:
        raise ValueError(
            f"Arg `pvalues` should have 1 element (for depolarizing noise: p) or "
            f"3 elements (for asymmetric depolarizing noise: px, py, pz), but "
            f"`len(pvalues)` was {len(pvalues)}."
        )

    gs_stb = stab_state.gs
    ps_stb = stab_state.ps
    N = (gs_stb.shape[1]) // 2
    r = stab_state.r
    # sample uncorrelated X,Y,Z gate
    tmp = {0: [0, 0], 1: [1, 0], 2: [1, 1], 3: [0, 1]}
    elements = [0, 1, 2, 3]
    probabilities = [1 - (px + py + pz), px, py, pz]
    sam = np.random.choice(elements, N, p=probabilities)
    gs_obs = (np.array([tmp[i] for i in sam]).flatten()).reshape((1, -1))
    new_gs, new_ps, new_r = pauli_operation(gs_stb, ps_stb, gs_obs, r)
    new_state = stabilizer.StabilizerState(new_gs, new_ps)
    new_state.set_r(new_r)
    return new_state

## Experiment

In [ ]:
"""Set experimental parameters."""
N = 5  # TODO: Get this from the stabilizer code.
expr_num = 20
channel_sample = 1_000

# Depolarizing noise rates.
pvalues = np.array([0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5])

In [ ]:
"""Run the experiment."""
fid_list = np.ones_like(pvalues)

for (i, p) in enumerate(pvalues):
    # Status update.
    print("Status: On noise rate", p, end="\r")

    Tr_P_sigma_P_rho = []
    Tr_P_sigma_P = []
    for _ in range(expr_num):
        for _ in range(channel_sample):
            # Initial stabilizer state.
            state = stabilizer_state("ZZZZZ", "XZZXI", "IXZZX", "XIXZZ", "ZXIXZ")
            gs0 = state.gs.copy()
            ps0 = state.ps.copy()

            # Apply single-qubit depolarizing noise.
            state = stoc_depolarize_map(state, p)

            # Do shadow tomography.
            obs = random_clifford_state(5)
            state.measure(obs)

            gs, ps, _, tmp_PsigmaP = utils.stabilizer_projection_full(
                state.gs, state.ps, gs0[: N - 1].copy(), ps0[: N - 1].copy(), 0
            )
            _, _, _, tmp = utils.stabilizer_projection_full(gs, ps, gs0[:N], ps0[:N], 0)
            Tr_P_sigma_P_rho.append(tmp_PsigmaP * tmp)
            Tr_P_sigma_P.append(tmp_PsigmaP)

    # Compute final fidelity.
    fid = ((2 ** N + 1) * np.mean(Tr_P_sigma_P_rho) - 1) / (
        np.mean(Tr_P_sigma_P) * (2 ** N + 1) - 2
    )
    fid_list[i] = fid

In [ ]:
def FL(p):
    N = (8 * p ** 2 - 9 * p + 3) * (2 * p - 3) ** 3
    D = 240 * p ** 4 - 720 * p ** 3 + 810 * p ** 2 - 405 * p + 81
    return -N / D

In [ ]:
plt.plot(pvalues, 1 - fid_list, "^", markersize=8, label="LST")
plt.plot(pvalues, 2 * pvalues / 3, "-.", c="k", label="Physical")
plt.plot(
    np.linspace(0, 0.5, 50), 1 - FL(np.linspace(0, 0.5, 50)), c="C3", label="Analytical"
)

plt.xlabel("$p$")
plt.ylabel("$1 - F$")
plt.legend();